## Get flight loads from SFO
### Prereq:
- Selenium driver
- Chrome driver
- .config.yaml with your Fly credentials

        fly:
          username: #######
          pw: xxxxxx

- csv file with list of origins
- csv file with list of destinations

In [1]:
## Need to install selenium driver and chrome driver
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import yaml
import csv

In [2]:
config = yaml.safe_load(open("./.config.yml"))

In [3]:
def fly_login(username, password):
    # head to fly login page
    driver = webdriver.Chrome("chromedriver")
    driver.get("https://fly.alaskasworld.com/")
    # find username/email field and send the username itself to the input field
    driver.find_element_by_id("UserName").send_keys(username)
    # find password input field and insert password as well
    driver.find_element_by_id("Password").send_keys(password)
    # click login button
    driver.find_element_by_id("loginsubmit").click()
    return driver

In [4]:
def connecting_flights(driver):
    num_layover = len(driver.find_elements_by_xpath('//*[contains(@class,"flight-divider")]'))
    return (num_layover > 0)

In [5]:
def clean_flight_list(flight_list):
    flight_list['orig'] = flight_list['orig_and_time'].str[:3]
    flight_list['dep'] = pd.to_datetime(flight_list['orig_and_time'].str[4:], format='%I:%M %p').dt.strftime('%H:%M')
    flight_list['dest'] = flight_list['dest_and_time'].str[:3]
    flight_list['arr'] = pd.to_datetime(flight_list['dest_and_time'].str[4:], format='%I:%M %p').dt.strftime('%H:%M')
    return flight_list.drop(columns=['orig_and_time','dest_and_time','show_details'])

In [6]:
def get_flight_list(driver):
    table = driver.find_elements_by_xpath('//*[contains(@class,"flight-details")]')
    flight_list = pd.DataFrame(columns=['flight_num','orig_and_time','dest_and_time','config','flight_time','show_details'])
    for t in table:
        flight_list.loc[len(flight_list)] = t.text.split('\n')
    return clean_flight_list(flight_list)

In [7]:
def clean_flight_seats(flight_seat_list):
    flight_seats = pd.DataFrame(columns=['config_f','config_open_f','config_y','config_open_y','auth_f','auth_open_f','auth_y','auth_open_y', 'listed'])
    for fs in flight_seat_list:
        if fs[0] == 'Cnfg':
            # Next flight - initiate a row
            flight_seats.loc[len(flight_seats)] = None
        if fs[0] in ['Cnfg','Auth','Cnfg Open','Auth Open']:
            first_number = fs[0].lower().replace('cnfg','config').replace(' ','_')+'_f'
            second_number = fs[0].lower().replace('cnfg','config').replace(' ','_')+'_y'
            flight_seats.loc[len(flight_seats)-1][first_number] = fs[1]
            flight_seats.loc[len(flight_seats)-1][second_number] = fs[2]
        elif fs[0] == 'Listed':
            flight_seats.loc[len(flight_seats)-1]['listed'] = fs[1]
    return flight_seats

In [8]:
def get_flight_seats(driver):
    flight_seat_list = []
    for table in driver.find_elements_by_xpath('//*[contains(@id,"flightSeatDetails")]//tr'):
        data = [item.get_attribute("innerHTML") for item in table.find_elements_by_xpath(".//*[self::td or self::th]")]
        flight_seat_list.append(data)
    return clean_flight_seats(flight_seat_list)

In [9]:
def get_flight_details(orig, dest, flight_date):
    url = "https://fly.alaskasworld.com/search/{0}/{1}/{2}".format(orig, dest, flight_date)
    driver.get(url)
    if connecting_flights(driver):
        return None
    else:
        flight_list = get_flight_list(driver)
        flight_seats = get_flight_seats(driver)
        flight_data = pd.concat([flight_list,flight_seats], axis=1)
        return flight_data

In [10]:
def get_airports(filename):
    with open("./"+filename) as f:
        reader = csv.reader(f)
        return list(reader)

## Pass in date and file names here

Results will be in the output file when it's done running

In [11]:
flight_date = '07Mar'
## Note: One of these files should only have 1 airport to limit the # calls!!!
dest_file = 'sfo_orig.csv'
orig_file = 'sfo_dest.csv'
output_file = 'flight_results_{0}.csv'.format(flight_date)


username = config['fly']['username']
pw = config['fly']['pw']
driver = fly_login(username, pw)
orig_list = get_airports(orig_file)
dest_list = get_airports(dest_file)
final_results = pd.DataFrame()
for o in orig_list:
    orig = o[0]
    for d in dest_list:
        dest = d[0]
        results = get_flight_details(orig,dest,flight_date)
        #print(results)
        final_results = final_results.append(results)
final_results.to_csv(output_file)